In [0]:
%sql
CREATE OR REPLACE TABLE cleaned_vessels AS
SELECT
  vessel_id,
  imo,
  vessel_name,
  vessel_status,
  vessel_status_group,
  age_in_year
FROM rightship_test.default.qa_tht_sample_data_model_vessels
WHERE vessel_id > 0
  AND imo IS NOT NULL;

SELECT * FROM cleaned_vessels;

vessel_id,imo,vessel_name,vessel_status,vessel_status_group,age_in_year
503072,2950038197,Sea Horizon,In Trading Fleet,In Service,46
489572,7645524261,Aqua Nova,In Trading Fleet,In Service,17
234602,6847158436,Starboard Spirit,Total Loss,Not In Service,38
125774,4807079427,Iron Gull,In Trading Fleet,In Service,4
175656,4312947302,Storm Petrel,Total Loss,Not In Service,75
146842,2438466352,Silver Dolphin,In Trading Fleet,In Service,34
122906,5779842756,Kraken's Shadow,In Trading Fleet,In Service,10
420160,2358534653,Poseidon's Arrow,In Trading Fleet,In Service,37
200557,8336538328,Neptune's Crown,In Trading Fleet,In Service,33


1. Unit testing

In [0]:
%sql
-- Unit test - check for duplicate imo
SELECT imo, COUNT(*) AS imo_count
FROM cleaned_vessels
GROUP BY imo
HAVING COUNT(*) > 1;

imo,imo_count


In [0]:
%sql
-- count voyage by imo
SELECT imo, COUNT(*) AS voyage_count
FROM rightship_test.default.qa_tht_sample_data_model_daily_voyage
GROUP BY imo
HAVING COUNT(*) >= 1;

imo,voyage_count
5779842756,3
1000069,17
4807079427,15
1000239,13
1000253,1


In [0]:
%sql
-- last arrival date by imo
SELECT
  imo,
  MAX(crew_entered_eta) AS last_voyage_time
FROM rightship_test.default.qa_tht_sample_data_model_daily_voyage
GROUP BY imo;

imo,last_voyage_time
5779842756,2024-08-13T18:00:00.000Z
1000069,2025-09-02T06:00:00.000Z
4807079427,2025-09-27T19:00:00.000Z
1000239,2025-09-15T17:30:00.000Z
1000253,2025-01-01T01:45:00.000Z


2. Integration testing

In [0]:
%sql
-- Join with voyage data to validate relationships then count voyage by imo
CREATE OR REPLACE TABLE vessels_data_integration_test AS
SELECT
  v.imo,
  COUNT(dv.imo) AS voyage_count
FROM cleaned_vessels AS v
LEFT JOIN rightship_test.default.qa_tht_sample_data_model_daily_voyage AS dv
  ON v.imo = dv.imo
GROUP BY v.imo;

SELECT * FROM vessels_data_integration_test;


imo,voyage_count
4312947302,0
7645524261,0
2950038197,0
2438466352,0
6847158436,0
2358534653,0
4807079427,15
5779842756,3
8336538328,0


In [0]:
%sql
-- imo is available on daily voyages but not on vessels
SELECT DISTINCT dv.imo
FROM rightship_test.default.qa_tht_sample_data_model_daily_voyage AS dv
LEFT JOIN cleaned_vessels AS v
  ON dv.imo = v.imo
WHERE v.imo IS NULL;


imo
1000069
1000239
1000253


3. System testing

In [0]:
%sql
--  Active vessels with invalid status group
SELECT DISTINCT
  v.imo,
  v.vessel_status_group
FROM cleaned_vessels AS v
JOIN rightship_test.default.qa_tht_sample_data_model_daily_voyage dv
  ON v.imo = dv.imo
WHERE v.vessel_status_group = 'Not In Service';

imo,vessel_status_group


In [0]:
%sql
-- TEST: Active vessels should have voyage history
SELECT
  v.imo,
  v.vessel_status_group,
  MAX(dv.crew_entered_eta) AS last_voyage_time
FROM cleaned_vessels AS v
LEFT JOIN rightship_test.default.qa_tht_sample_data_model_daily_voyage AS dv
  ON v.imo = dv.imo
WHERE v.vessel_status_group = 'In Service'
GROUP BY v.imo, v.vessel_status_group
HAVING MAX(dv.crew_entered_eta) IS NULL;


imo,vessel_status_group,last_voyage_time
2358534653,In Service,null
2438466352,In Service,null
2950038197,In Service,null
7645524261,In Service,null
8336538328,In Service,null


4. Regression testing

In [0]:
%sql
-- REGRESSION TEST: vessel count consistency
SELECT COUNT(*) AS vessel_count
FROM cleaned_vessels;


vessel_count
9


In [0]:
%sql
SELECT COUNT(DISTINCT imo) AS voyage_imo_count
FROM rightship_test.default.qa_tht_sample_data_model_daily_voyage;

voyage_imo_count
5


4. System testing

In [0]:
%sql
SELECT COUNT(*) AS active_vessels_without_voyage
FROM (
  SELECT v.imo
  FROM cleaned_vessels v
  LEFT JOIN rightship_test.default.qa_tht_sample_data_model_daily_voyage dv
    ON v.imo = dv.imo
  WHERE v.vessel_status_group = 'In Service'
  GROUP BY v.imo
  HAVING MAX(dv.crew_entered_eta) IS NULL);


active_vessels_without_voyage
5


In [0]:
%sql
-- Check vessel status for IMOs with voyage history
SELECT DISTINCT
  v.imo,
  v.vessel_status,
  v.vessel_status_group
FROM cleaned_vessels AS v
JOIN rightship_test.default.qa_tht_sample_data_model_daily_voyage AS dv
  ON v.imo = dv.imo;


imo,vessel_status,vessel_status_group
5779842756,In Trading Fleet,In Service
4807079427,In Trading Fleet,In Service
